# RL and Advanced DL: Домашнее задание 2



In [1]:
%autosave 60

Autosaving every 60 seconds


In [2]:
import gym
import sys
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import json
import math
import random
import numpy as np
import scipy as sp
import scipy.stats as st
import scipy.integrate as integrate
from scipy.stats import multivariate_normal
from sklearn import linear_model
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import statsmodels.api as sm
from matplotlib.colors import LogNorm
import pickle

from joblib import Parallel, delayed
import multiprocessing
from collections import namedtuple
from itertools import count
from itertools import product
import cProfile
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.autograd import Variable

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('figure', **{'dpi': 300})

## Классы для крестиков ноликов

In [3]:
N_ROWS, N_COLS, N_WIN = 3, 3, 3

In [4]:
class TicTacToe(gym.Env):
    def __init__(self, n_rows=N_ROWS, n_cols=N_COLS, n_win=N_WIN, clone=None):
        if clone is not None:
            self.n_rows, self.n_cols, self.n_win = clone.n_rows, clone.n_cols, clone.n_win
            self.board = copy.deepcopy(clone.board)
            self.curTurn = clone.curTurn
            self.emptySpaces = None
            self.boardHash = None
        else:
            self.n_rows = n_rows
            self.n_cols = n_cols
            self.n_win = n_win

            self.reset()

    def getEmptySpaces(self):
        if self.emptySpaces is None:
            res = np.where(self.board == 0)
            self.emptySpaces = np.array([ (i, j) for i,j in zip(res[0], res[1]) ])
        return self.emptySpaces

    def makeMove(self, player, i, j):
        self.board[i, j] = player
        self.emptySpaces = None
        self.boardHash = None

    def getHash(self):
        if self.boardHash is None:
            self.boardHash = ''.join(['%s' % (x+1) for x in self.board.reshape(self.n_rows * self.n_cols)])
        return self.boardHash

    def isTerminal(self):
        # проверим, не закончилась ли игра
        cur_marks, cur_p = np.where(self.board == self.curTurn), self.curTurn
        for i,j in zip(cur_marks[0], cur_marks[1]):
            win = False
            if i <= self.n_rows - self.n_win:
                if np.all(self.board[i:i+self.n_win, j] == cur_p):
                    win = True
            if not win:
                if j <= self.n_cols - self.n_win:
                    if np.all(self.board[i,j:j+self.n_win] == cur_p):
                        win = True
            if not win:
                if i <= self.n_rows - self.n_win and j <= self.n_cols - self.n_win:
                    if np.all(np.array([ self.board[i+k,j+k] == cur_p for k in range(self.n_win) ])):
                        win = True
            if not win:
                if i <= self.n_rows - self.n_win and j >= self.n_win-1:
                    if np.all(np.array([ self.board[i+k,j-k] == cur_p for k in range(self.n_win) ])):
                        win = True
            if win:
                self.gameOver = True
                return self.curTurn

        if len(self.getEmptySpaces()) == 0:
            self.gameOver = True
            return 0

        self.gameOver = False
        return None

    def printBoard(self):
        for i in range(0, self.n_rows):
            print('----'*(self.n_cols)+'-')
            out = '| '
            for j in range(0, self.n_cols):
                if self.board[i, j] == 1:
                    token = 'x'
                if self.board[i, j] == -1:
                    token = 'o'
                if self.board[i, j] == 0:
                    token = ' '
                out += token + ' | '
            print(out)
        print('----'*(self.n_cols)+'-')

    def getState(self):
        return (self.getHash(), self.getEmptySpaces(), self.curTurn)

    def action_from_int(self, action_int):
        return ( int(action_int / self.n_cols), int(action_int % self.n_cols))

    def int_from_action(self, action):
        return action[0] * self.n_cols + action[1]
    
    def step(self, action):
        if self.board[action[0], action[1]] != 0:
            return self.getState(), -10, True, {}
        self.makeMove(self.curTurn, action[0], action[1])
        reward = self.isTerminal()
        self.curTurn = -self.curTurn
        return self.getState(), 0 if reward is None else reward, reward is not None, {}

    def reset(self):
        self.board = np.zeros((self.n_rows, self.n_cols), dtype=int)
        self.boardHash = None
        self.gameOver = False
        self.emptySpaces = None
        self.curTurn = 1

In [5]:
def plot_board(env, pi, showtext=True, verbose=True, fontq=20, fontx=60):
    '''Рисуем доску с оценками из стратегии pi'''
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    X, Y = np.meshgrid(np.arange(0, env.n_rows), np.arange(0, env.n_rows))
    Z = np.zeros((env.n_rows, env.n_cols)) + .01
    s, actions = env.getHash(), env.getEmptySpaces()
    if pi is not None and s in pi.Q:
        for i, a in enumerate(actions):
            Z[a[0], a[1]] = pi.Q[s][i]
    ax.set_xticks([])
    ax.set_yticks([])
    surf = ax.imshow(Z, cmap=plt.get_cmap('Accent', 10), vmin=-1, vmax=1)
    if showtext:
        for i,a in enumerate(actions):
            if pi is not None and s in pi.Q:
                ax.text( a[1] , a[0] , "%.3f" % pi.Q[s][i], fontsize=fontq, horizontalalignment='center', verticalalignment='center', color="w" )
    for i in range(env.n_rows):
        for j in range(env.n_cols):
            if env.board[i, j] == -1:
                ax.text(j, i, "O", fontsize=fontx, horizontalalignment='center', verticalalignment='center', color="w" )
            if env.board[i, j] == 1:
                ax.text(j, i, "X", fontsize=fontx, horizontalalignment='center', verticalalignment='center', color="w" )
#     cbar = plt.colorbar(surf, ticks=[0, 1])
    ax.grid(False)
    plt.show()

def get_and_print_move(env, pi, s, actions, random=False, verbose=True, fontq=20, fontx=60):
    '''Делаем ход, рисуем доску'''
    plot_board(env, pi, fontq=fontq, fontx=fontx)
    if verbose and (pi is not None):
        if s in pi.Q:
            for i,a in enumerate(actions):
                print(i, a, pi.Q[s][i])
        else:
            print("Стратегия не знает, что делать...")
    if random:
        return np.random.randint(len(actions))
    else:
        return pi.getActionGreedy(s, len(actions))


# Табличный Q-learning


## Идея: сделаем два агента, которые будут играть друг против друга по своим табличным функциям 


In [352]:
class SexyEnvironment():
    def __init__(self,base_env):
        
        self.env = base_env
        self.env.reset()
        
        self.action_space_dim = len(self.env.getState()[1])
        self.state_space_dim = 3** len(self.env.getState()[0])
        self.state_spaces = len(self.env.getState()[0])
        
        # self.state_to_int={v:k for k,v in enumerate(product('012', repeat=self.state_space_dim))}
        # self.state = self._get_state()
        
    def reset(self):
        self.env.reset()
        return tuple(map(int,self.env.getState()[0]))
    @property
    def state(self):
        return tuple(map(int,self.env.getState()[0]))
        
    def step(self,action):
        env_output = self.env.step(self.env.action_from_int(action))
        state= tuple(map(int,env_output[0][0]))
        reward = env_output[1]
        terminated = env_output[2]
        # print('env',env_output,self.state)
        if reward ==-1:
            reward=1
        return state,reward,terminated

In [353]:
class TableQlearningAgent():
    def __init__(self,action_space_dim,state_spaces,agent_name,alpha=0.01,gamma=0.3):
        self.Q = np.random.uniform(size=(tuple([3 for _ in range(state_spaces)])+(action_space_dim,)))
        self.max_learning_iterations = 10_000
        self.alpha = alpha
        self.gamma = gamma
        self.agent_name = agent_name
        self.epsilon = 0.1
        
        self.counter = 1
        self.Q_prev = None
        
    
        
        self.rewards = []
        
        
    def get_action(self, s):
        
        if np.random.uniform() < 1-self.epsilon:
            return np.argmax(self.Q[s])
        else:
            return np.random.choice(list(range(len(self.Q[s]))))
        
    def one_step(self,env):
        
        s_prev = env.state
        self.Q_prev=self.Q.copy()

        action=self.get_action(s_prev)
        s,reward,terminate=env.step(action)
        
        
        self.rewards.append(reward)
        self.Q[s_prev][action] = self.Q[s_prev][action]+self.alpha*(reward+self.gamma*np.argmax(self.Q[s]) - self.Q[s_prev][action])
        
        self.counter+=1
    
        return reward,terminate
    
    
        
    
        
        
        
        

In [357]:
def Q_learn(agent_cross,agent_zeros,env,num_games=10):
    is_cross_queue = True
    counter = 1
    cross_reward_by_game = []
    zeros_reward_by_game= []
    
    for _ in range(num_games):
        env.reset()
       
        cross_rewards = []
        zeros_rewards = []
        terminate=False
        while not terminate:
        
            if is_cross_queue:
                reward,terminate = agent_cross.one_step(env)
                cross_rewards.append(reward)
                is_cross_queue = False
            else:
                reward,terminate = agent_zeros.one_step(env)
                zeros_rewards.append(reward)
                is_cross_queue = True
                
            # print(is_cross_queue,reward)
            if terminate:
                break
            # print(env.env.printBoard())
        
        
        cross_reward_by_game.append(np.mean(cross_rewards))
        zeros_reward_by_game.append(np.mean(zeros_rewards))
        if _%1000==0:
            print(f'Game #{_} Cross reward {cross_reward_by_game[_]:.3f} Zeros reward {zeros_reward_by_game[_]:.3f}')
                
        

In [358]:
other_env = TicTacToe(n_rows=3, n_cols=3, n_win=3)

In [359]:
env = SexyEnvironment(other_env)

In [360]:
agent_cross = TableQlearningAgent(action_space_dim=env.action_space_dim,state_spaces=env.state_spaces,agent_name='Cross')

agent_zeros = TableQlearningAgent(action_space_dim=env.action_space_dim,state_spaces=env.state_spaces,agent_name='Zeros')

In [ ]:
Q_learn(agent_cross,agent_zeros,env,num_games=1000000)

Game #0 Cross reward -5.000 Zeros reward 0.000
Game #1000 Cross reward 0.000 Zeros reward 0.200
Game #2000 Cross reward -2.500 Zeros reward 0.000
Game #3000 Cross reward 0.000 Zeros reward 0.200
Game #4000 Cross reward 0.250 Zeros reward 0.000
Game #5000 Cross reward 0.250 Zeros reward 0.000
Game #6000 Cross reward 0.250 Zeros reward 0.000
Game #7000 Cross reward 0.333 Zeros reward 0.000
Game #8000 Cross reward 0.333 Zeros reward 0.000
Game #9000 Cross reward 0.000 Zeros reward -2.500
Game #10000 Cross reward -2.500 Zeros reward 0.000
Game #11000 Cross reward 0.000 Zeros reward 0.200
Game #12000 Cross reward 0.000 Zeros reward 0.200
Game #13000 Cross reward -3.333 Zeros reward 0.000
Game #14000 Cross reward 0.000 Zeros reward 0.200
Game #15000 Cross reward -5.000 Zeros reward 0.000
Game #16000 Cross reward 0.000 Zeros reward -5.000
Game #17000 Cross reward 0.000 Zeros reward 0.250
Game #18000 Cross reward 0.000 Zeros reward 0.250
Game #19000 Cross reward 0.200 Zeros reward 0.000
Game #